# Skeleton Simulation

#### Imports

In [1]:
import sys
import math
import warnings

import psycopg2
import wrds
import gzip

import seaborn as sns
import os
import quandl
import json
import zipfile
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import functools
import requests
import io

import urllib.request
from urllib.error import HTTPError
from html_table_parser.parser import HTMLTableParser
from bs4 import BeautifulSoup
import re

import plotnine as p9
from plotnine import ggplot, scale_x_date, guides, guide_legend, geom_bar, scale_y_continuous, \
    scale_color_identity, geom_line, geom_point, labs, theme_minimal, theme, element_blank, element_text, \
        geom_ribbon, geom_hline, aes, scale_size_manual, scale_color_manual, ggtitle

from datetime import datetime
import datetime

import pandas as pd
import pandas_market_calendars as mcal
from pandas.plotting import autocorrelation_plot
import numpy as np
from numpy import cumsum, log, polyfit, sqrt, std, subtract
import scipy as sp
from scipy.stats import norm
import scipy.stats as stats

from statsmodels.tsa.stattools import coint
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant

from collections import deque
from bisect import insort, bisect_left
from itertools import islice


### Retrieve data

In [2]:
csv_file_path = 'combinedata.csv'
data = pd.read_csv(csv_file_path)

In [3]:
display(data)

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,expiry_indicator,close,adj_open,adj_close,adj_volume
0,2018-01-02,2018-01-03,2017-12-28,C,235.0,33.59,33.81,0.0,187.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0
1,2018-01-02,2018-01-03,2018-01-02,C,240.0,28.59,28.76,1.0,88.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0
2,2018-01-02,2018-01-03,2017-12-27,C,242.5,26.09,26.32,0.0,2.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0
3,2018-01-02,2018-01-03,2018-01-02,C,245.0,23.59,23.81,12.0,58.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0
4,2018-01-02,2018-01-03,NaN,C,247.5,21.08,21.32,0.0,0.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10448189,2023-02-28,2025-03-21,2022-12-28,P,600.0,200.50,205.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,396.26,391.240559,390.285185,96141367.0
10448190,2023-02-28,2025-03-21,NaN,P,605.0,205.50,210.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,396.26,391.240559,390.285185,96141367.0
10448191,2023-02-28,2025-03-21,NaN,P,610.0,210.00,215.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,396.26,391.240559,390.285185,96141367.0
10448192,2023-02-28,2025-03-21,NaN,P,615.0,215.00,220.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,396.26,391.240559,390.285185,96141367.0


### Simulation

This simulation code simulates trading with respect to pairs of individual contracts, with an initial delta of as close to 1 as possible.

The option contracts are opened at varying expiries, and are held until 1 week before expiry. 

Position in SPY is taken daily to rebalance the option delta as the underlying moves, and P&L is tracked from the option premium and SPY rebalances.
- Obtain the zero-delta pairs, usually somewhat symmetrical about the ATM strike (how many do we want)
- Initial balance of stock for the remaining delta
- Every day, we will rebalance the position according to delta. 
- Save the daily position, and calculate daily PL vectorized.

Other considerations and parameters are described in ```outline.md```.

In [ ]:
class Simulate:
    pass